# "Covid-19 ~ India"
> "Some charts exploring Covid-19 India Data"

- toc: false 
- badges: false
- comments: true
- categories: [ai-in-society]
- image: images/Covid 19.jpeg
- permalink: /covid-19/

>Note: The following charts are based on official Government Of India sources.

In [1]:
#hide
from IPython.display import HTML, Javascript, display
from string import Template
import numpy as np
import pandas as pd
import json
import math
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib as mpl
import time
import requests
from datetime import datetime

In [2]:
#hide
mpl.rcParams['figure.dpi'] = 100
display(Javascript("require.config({paths: {d3: 'https://d3js.org/d3.v5.min.js'}});"));

<IPython.core.display.Javascript object>

In [3]:
#hide
"""Simple script to print HTML style markdown text in a notebook"""
from IPython.display import Markdown, display

class BeautifulText():
    """
    Display HTML format Markdown text in the notebook. Useful for highlighting important notes in
    the notebook.
    """
    def __init__(self, color: str = 'black', font_family: str = 'Arial',
                 font_weight: (str, int) = 'normal', font_size: int = 16, font_style: str = 'normal',
                 background_color: str = 'none', letter_spacing: int = 1, line_height: float = 0.8,
                 word_spacing: int = 1, text_decoration: str = 'none', text_shadow: str = 'none'):
        """
        Initialise with HTML style elements.
        Parameters
        ----------
        color : str
            Color of the markdown text. Defaults to `black`.
        font_family : str
            `font-family` html attribute. Defaults to `serif`.
        font_weight : str
            `font-weight` html attribute. Defaults to `normal`.
        font_size : int
            `font-size` html attribute. Defaults to `16`px. (Don't include `px` in the argument.)
        font_style : str
            `font-style` html attribute. Defaults to `normal`.
        background_color : str
            `background-color` html attribute. Defaults to `none`
        letter_spacing : int
            `letter-spacing` html attribute. Defaults to `1`px.
            (Don't include `px` in the argument.)
        line_height : float
            `line-height` html attribute. Defaults to `0.8`px. (Don't include `px` in the argument.)
        word_spacing : int
            `word-spacing` html attribute. Defaults to `1`px. (Don't include `px` in the argument.)
        text-decoration : str
            `text-decoration` html attribute. Defaults to `none`.
        text-shadow : str
            `text-shadow` html attribute. Defaults to `none`.
        """
        self.color = color
        self.font_family = font_family
        self.font_weight = font_weight
        self.font_size = font_size
        self.font_style = font_style
        self.background_color = background_color
        self.letter_spacing = letter_spacing
        self.line_height = line_height
        self.word_spacing = word_spacing
        self.text_decoration = text_decoration
        self.text_shadow = text_shadow
    def printbeautiful(self, text: str) -> None:
        """Applies the html attributes to `text` and prints it as MarkDown."""
        beautifultext = f"""<span style='color: {self.color};
                        font-family: {self.font_family};
                        font-weight: {self.font_weight};
                        font-size: {self.font_size}px;
                        font-style: {self.font_style};
                        text-decoration: {self.text_decoration};
                        letter-spacing: {self.letter_spacing}px;
                        line-height: {self.line_height}px;
                        word-spacing: {self.word_spacing}px;
                        text-shadow: {self.text_shadow};
                        background-color: {self.background_color};'>
                        {text}
                        </span>
                        """
        display(Markdown(beautifultext))


In [4]:
#hide
commentary = BeautifulText(font_family="Helvetica Neue", font_size=20, line_height=30, color="#515151", font_weight=400)

In [5]:
#hide
css_text = '''
.main-wrapper {
    position: relative;
}

.line {
    fill: none;
    stroke: #FF9933;
    stroke-width: 2;
}

.y-axis-label {
    fill: black;
    font-size: 1.4em;
    text-anchor: middle;
    transform: rotate(-90deg);
}


.main-listening-rect {
    fill: transparent;
}

.listening-rect {
    fill: transparent;
}

.chartcon {
    display: flex;
    justify-content: center;
    padding: 5em 1em;
    font-family: sans-serif;
}

.main-tooltip {
    opacity: 0;
    position: absolute;
    top: -14px;
    left: 0;
    padding: 0.6em 1em;
    background: #fff;
    text-align: center;
    line-height: 1.4em;
    font-size: 0.9em;
    border: 1px solid #ddd;
    z-index: 10;
    transition: all 0.1s ease-out;
    pointer-events: none;
}

.tooltip {
    opacity: 0;
    position: absolute;
    top: -14px;
    left: 0;
    padding: 0.6em 1em;
    background: #fff;
    text-align: center;
    line-height: 1.4em;
    font-size: 0.9em;
    border: 1px solid #ddd;
    z-index: 10;
    transition: all 0.1s ease-out;
    pointer-events: none;
}

.main-tooltip:before {
    content: '';
    position: absolute;
    bottom: 0;
    left: 50%;
    width: 12px;
    height: 12px;
    background: white;
    border: 1px solid #ddd;
    border-top-color: transparent;
    border-left-color: transparent;
    transform: translate(-50%, 50%) rotate(45deg);
    transform-origin: center center;
    z-index: 10;
}

.tooltip:before {
    content: '';
    position: absolute;
    bottom: 0;
    left: 50%;
    width: 12px;
    height: 12px;
    background: white;
    border: 1px solid #ddd;
    border-top-color: transparent;
    border-left-color: transparent;
    transform: translate(-50%, 50%) rotate(45deg);
    transform-origin: center center;
    z-index: 10;
}

.main-tooltip-date {
    margin-bottom: 0.2em;
    font-weight: 600;
    font-size: 1.1em;
    line-height: 1.4em;
}

.tooltip-date {
    margin-bottom: 0.2em;
    font-weight: 600;
    font-size: 1.1em;
    line-height: 1.4em;
}

.rate-wrapper {
    position: relative;
}
    
.rate-line {
    fill: none;
    stroke: #8B0000;
    stroke-width: 2;
}

.summary-wrapper {
    position: relative;
}

.summary-confirmed{
    fill: #8B0000;

}

.confirmed-badge{
    fill : white;
    font-size: 13pt;
    font-weight: 700;
    text-anchor: middle;
    font-family: Arial, Helvetica, sans-serif;
}

.confirmed-badge-value{
    fill : white;
    font-size: 38pt;
    font-weight: 700;
    text-anchor: middle;
    padding: 10px;
}

.summary-active{
    fill: #FF8C00;

}

.active-badge{
    fill : white;
    font-size: 13pt;
    font-weight: 700;
    text-anchor: middle;
    font-family: Arial, Helvetica, sans-serif;
}

.active-badge-value{
    fill : white;
    font-size: 38pt;
    font-weight: 700;
    text-anchor: middle;
    padding: 10px;
}

.summary-recovered{
    fill: #006400;

}

.recovered-badge{
    fill : white;
    font-size: 13pt;
    font-weight: 700;
    text-anchor: middle;
    font-family: Arial, Helvetica, sans-serif;
}

.recovered-badge-value{
    fill : white;
    font-size: 38pt;
    font-weight: 700;
    text-anchor: middle;
    padding: 10px;
}

.summary-deceased{
    fill: #A9A9A9;

}

.deceased-badge{
    fill : white;
    font-size: 13pt;
    font-weight: 700;
    text-anchor: middle;
    font-family: Arial, Helvetica, sans-serif;
}

.deceased-badge-value{
    fill : white;
    font-size: 38pt;
    font-weight: 700;
    text-anchor: middle;
    padding: 10px;
}

'''

## Summary

In [6]:
#hide
official_latest_url = "https://api.rootnet.in/covid19-in/stats/latest"
latest_raw = requests.get(official_latest_url).text
latest = json.loads(latest_raw)['data']

In [7]:
#hide
confirmed_count = latest['summary']['total']
recovered_count = latest['summary']['discharged']
deceased_count = latest['summary']['deaths']
active_count = confirmed_count - (recovered_count + deceased_count)

In [8]:
#hide
summary_js = f"""
let confirmed = {confirmed_count}
let recovered = {recovered_count}
let deceased = {deceased_count}
let active = {active_count}
"""

In [9]:
#hide
summary_html_temp = Template('''
    <script src = "https://d3js.org/d3.v5.min.js"></script> 
    <style scoped>
        $css_text
    </style>
    <div id="summary-wrapper" class="summary-wrapper">
        </div>    
    <script>
    $summary_data
    $d3_script_summary
    </script>
''')

In [10]:
#hide
d3_script_summary = '''
function drawSummaryChart() {

  let dimensions = {
    width: 750,
    height: 100,
  }

  // 3. Draw canvas

  const summary_wrapper = d3.select("#summary-wrapper")
    .append("svg")
    .attr("preserveAspectRatio", "xMinYMin meet")
    .attr("viewBox", "0 0 750 100")
    //.attr("width", dimensions.width)
    //.attr("height", dimensions.height)

  const confirmed_badge = summary_wrapper.append("rect")
    .attr("class", "summary-confirmed")
    .attr("width", "20%")
    .attr("height", "100%")
    .attr("rx", 8)
   

  const active_badge = summary_wrapper.append("rect")
    .attr("class", "summary-active")
    .attr("x", (0.25 * dimensions.width))
    .attr("width", "20%")
    .attr("height", "100%")
    .attr("rx", 8)
   

  const recovered_badge = summary_wrapper.append("rect")
    .attr("class", "summary-recovered")
    .attr("x", (0.50 * dimensions.width))
    .attr("width", "20%")
    .attr("height", "100%")
    .attr("rx", 8)
    

  const deceased_badge = summary_wrapper.append("rect")
    .attr("class", "summary-deceased")
    .attr("x", (0.75 * dimensions.width))
    .attr("width", "20%")
    .attr("height", "100%")
    .attr("rx", 8)

  summary_wrapper.append("text")
  .attr("class", "confirmed-badge")
  .attr("x", (0.2 * dimensions.width)/ 2)
  .attr("y", (0.8 * dimensions.height))
  .text("Confirmed")

  summary_wrapper.append("text")
  .attr("class", "confirmed-badge-value")
  .attr("x", (0.2 * dimensions.width)/ 2)
  .attr("y", (0.5 * dimensions.height))
  .text(`${confirmed}`)

  summary_wrapper.append("text")
  .attr("class", "active-badge")
  .attr("x", (0.25 * dimensions.width) + (0.2 * dimensions.width)/ 2)
  .attr("y", (0.8 * dimensions.height))
  .text("Active")

  summary_wrapper.append("text")
  .attr("class", "active-badge-value")
  .attr("x", (0.25 * dimensions.width) +  (0.2 * dimensions.width)/ 2)
  .attr("y", (0.5 * dimensions.height))
  .text(`${active}`)

  summary_wrapper.append("text")
  .attr("class", "recovered-badge")
  .attr("x", (0.50 * dimensions.width) + (0.2 * dimensions.width)/ 2)
  .attr("y", (0.8 * dimensions.height))
  .text("Recovered")

  summary_wrapper.append("text")
  .attr("class", "active-badge-value")
  .attr("x", (0.50 * dimensions.width) +  (0.2 * dimensions.width)/ 2)
  .attr("y", (0.5 * dimensions.height))
  .text(`${recovered}`)

  summary_wrapper.append("text")
  .attr("class", "deceased-badge")
  .attr("x", (0.75 * dimensions.width) + (0.2 * dimensions.width)/ 2)
  .attr("y", (0.8 * dimensions.height))
  .text("Deceased")

  summary_wrapper.append("text")
  .attr("class", "deceased-badge-value")
  .attr("x", (0.75 * dimensions.width) +  (0.2 * dimensions.width)/ 2)
  .attr("y", (0.5 * dimensions.height))
  .text(`${deceased}`)
      
}
drawSummaryChart();
'''

In [11]:
#hide
summary_html_text = summary_html_temp.substitute({
    'css_text': css_text,
    'd3_script_summary' : d3_script_summary,
    'summary_data':summary_js
})

In [12]:
#hide_input
HTML(summary_html_text)

## Covid-19 India Timeline

The source data API{% fn 1 %} fetches data from March 10, 2020 onwards.

>Tip: Mouse over chart to see data points for a specific day.

In [13]:
#hide
official_history_url = "https://api.rootnet.in/covid19-in/stats/history" 
data_raw = requests.get(official_history_url).text

In [14]:
#hide
data = json.loads(data_raw)['data']

In [15]:
#hide
js_data = f'let dataset = {data}'

In [16]:
#hide
html_temp = Template('''
    <script src = "https://d3js.org/d3.v5.min.js"></script> 
    <style scoped>
        $css_text
    </style>
    <div id="main-wrapper" class="main-wrapper">
    <div id="main-tooltip" class="main-tooltip">
                <div class="main-tooltip-date">
                    <span id="date"></span>
                </div>
                <div class="main-confirmed">
                    Confirmed Cases: <span id="main-confirmed"></span>
                </div>
        </div>
    <script>
    $dataset
    $d3_script
    </script>
''')

In [17]:
#hide
d3_script = '''
function drawLineChart() {
  
  main_yAccessor = d => d.summary.total
  const dateParser = d3.timeParse("%Y-%m-%d")
  format = d3.timeFormat("%b-%d")
  const main_xAccessor = d => dateParser(d.day)
  dataset = dataset.sort((a,b) => main_xAccessor(a) - main_xAccessor(b)).slice(0, 100)

  let dimensions = {
    width: 750,
    height: 550,
    margin: {
      top: 20,
      right: 30,
      bottom: 30,
      left: 70,
    },
  }
  dimensions.boundedWidth = dimensions.width - dimensions.margin.left - dimensions.margin.right
  dimensions.boundedHeight = dimensions.height - dimensions.margin.top - dimensions.margin.bottom

  // 3. Draw canvas

  const main_wrapper = d3.select("#main-wrapper")
    .append("svg")
    .attr("width", dimensions.width)
    .attr("height", dimensions.height)
    
    main_wrapper.append("rect")
        .attr("class", "chart-background")
        .attr("width", "100%")
        .attr("height", "100%")
        .attr("fill", "#faebd7")

  const bounds = main_wrapper.append("g")
      .attr("transform", `translate(${
        dimensions.margin.left
      }, ${
        dimensions.margin.top
      })`)
      
    bounds.append("text")
      .attr("class", "chart-watermark")
      .attr("x", dimensions.width - 180)
      .attr("y", dimensions.height - 60)
      .attr("fill", "grey")
      .style("opacity", 0.5)
      .html("@Alephthoughts")

  bounds.append("defs").append("clipPath")
      .attr("id", "bounds-clip-path")
    .append("rect")
      .attr("width", dimensions.boundedWidth)
      .attr("height", dimensions.boundedHeight)

  const clip = bounds.append("g")
    .attr("clip-path", "url(#bounds-clip-path)")

  // 4. Create scales

  const main_yScale = d3.scaleLinear()
    .domain(d3.extent(dataset, main_yAccessor))
    .range([dimensions.boundedHeight, 0])
    .nice()

  
  const main_xScale = d3.scaleTime()
    .domain(d3.extent(dataset, main_xAccessor))
    .range([0, dimensions.boundedWidth])

  const lineGenerator = d3.line()
    .x(d => main_xScale(main_xAccessor(d)))
    .y(d => main_yScale(main_yAccessor(d)))

  const line = clip.append("path")
      .attr("class", "line")
      .attr("d", lineGenerator(dataset))

  const yAxisGenerator = d3.axisLeft()
      .scale(main_yScale)
      
  
  const yAxis = bounds.append("g")
      .attr("class", "y-axis")
      .call(yAxisGenerator)

  const yAxisLabel = yAxis.append("text")
      .attr("class", "y-axis-label")
      .attr("x", -dimensions.boundedHeight / 2)
      .attr("y", -dimensions.margin.left + 20)
      .html("Confirmed Cases")

      const xAxisGenerator = d3.axisBottom()
      .scale(main_xScale)
      .tickFormat(format)
      .ticks(7)
  
  const xAxis = bounds.append("g")
      .attr("class", "x-axis")
      .style("transform", `translateY(${dimensions.boundedHeight}px)`)
      .call(xAxisGenerator)

  const main_listeningRect = bounds.append("rect")
      .attr("class", "main-listening-rect")
      .attr("width", dimensions.boundedWidth)
      .attr("height", dimensions.boundedHeight)
      .on("mousemove", onMouseMove)
      .on("mouseleave", onMouseLeave)

  const main_tooltip = d3.select("#main-tooltip")
  const main_tooltipCircle = bounds.append("circle")
          .attr("class", "main-tooltip-circle")
          .attr("r", 4)
          .attr("stroke", "#FF9933")
          .attr("fill", "white")
          .attr("stroke-width", 2)
          .style("opacity", 0)

  function onMouseMove() {
    const mousePosition = d3.mouse(this)
    const xhoveredDate = main_xScale.invert(mousePosition[0])
        
    const main_getDistanceFromxhoveredDate = d => Math.abs(main_xAccessor(d) - xhoveredDate)
    const main_closestIndex = d3.scan(dataset, (a, b) => (
    main_getDistanceFromxhoveredDate(a) - main_getDistanceFromxhoveredDate(b)
    ))
    const main_closestDataPoint = dataset[main_closestIndex]
        
    const main_closestXValue = main_xAccessor(main_closestDataPoint)
    const main_closestYValue = main_yAccessor(main_closestDataPoint)
        
    const formatDate = d3.timeFormat("%A %B %-d, %Y")
    main_tooltip.select("#date")
    .text(formatDate(main_closestXValue))

    main_tooltip.select("#main-confirmed")
        .html(main_closestYValue)

    const x = main_xScale(main_closestXValue)
      + dimensions.margin.left
    const y = main_yScale(main_closestYValue)
      + dimensions.margin.top

    main_tooltip.style("transform", `translate(`
      + `calc(-50% + ${x}px),`
      + `calc(-120% + ${y}px)`
      + `)`)

    main_tooltip.style("opacity", 1)

    main_tooltipCircle
        .attr("cx", main_xScale(main_closestXValue))
        .attr("cy", main_yScale(main_closestYValue))
        .style("opacity", 1)
  }
  function onMouseLeave() {
    main_tooltip.style("opacity", 0)

    main_tooltipCircle.style("opacity", 0)
  }
      
}
drawLineChart();
'''

In [18]:
#hide
html_text = html_temp.substitute({
    'css_text': css_text,
    'd3_script' : d3_script,
    'dataset':js_data
})

In [19]:
#hide_input
HTML(html_text)

## Growth Rate Over Time

>Tip: Mouse over chart to see data points for a specific day.

In [20]:
#hide
day = ["2020-03-10"]
growth = [(7/40) * 100]
for i in range(1,len(data)):
    day.append(data[i]['day'])
    growth.append((data[i]['summary']['total'] - data[i-1]['summary']['total'])/data[i-1]['summary']['total'] * 100)

In [21]:
#hide
rate_data = json.dumps([{"day": day, "growth": growth} for day, growth in zip(day, growth)])

In [22]:
#hide
rate_data_js = f"let rate_dataset = {rate_data} "

In [23]:
#hide
rate_html_temp = Template('''
    <script src = "https://d3js.org/d3.v5.min.js"></script> 
    <style scoped>
        $css_text
    </style>
    <div id="rate-wrapper" class="rate-wrapper">

            <div id="tooltip" class="tooltip">
                <div class="tooltip-date">
                    <span id="date"></span>
                </div>
                <div class="tooltip-rate">
                    Rate: <span id="rate"></span>
                </div>
            </div>
        </div>
    <script>
    $rate_dataset
    $d3_rate_script
    </script>
''')

In [24]:
#hide
d3_rate_script = '''
function drawRateChart() {
  
  yAccessor = d => d.growth
  const dateParser = d3.timeParse("%Y-%m-%d")
  format = d3.timeFormat("%b-%d")
  const xAccessor = d => dateParser(d.day)
  rate_dataset = rate_dataset.sort((a,b) => xAccessor(a) - xAccessor(b)).slice(0, 100)

  let dimensions = {
    width: 750,
    height: 550,
    margin: {
      top: 20,
      right: 30,
      bottom: 30,
      left: 70,
    },
  }
  dimensions.boundedWidth = dimensions.width - dimensions.margin.left - dimensions.margin.right
  dimensions.boundedHeight = dimensions.height - dimensions.margin.top - dimensions.margin.bottom

  // 3. Draw canvas

  const rate_wrapper = d3.select("#rate-wrapper")
    .append("svg")
      .attr("width", dimensions.width)
      .attr("height", dimensions.height)

  rate_wrapper.append("rect")
    .attr("class", "background")
    .attr("width", "100%")
    .attr("height", "100%")
    .attr("fill", "#faebd7")

  const bounds = rate_wrapper.append("g")
      .attr("transform", `translate(${
        dimensions.margin.left
      }, ${
        dimensions.margin.top
      })`)

  

  // 4. Create scales

  const yScale = d3.scaleLinear()
    .domain(d3.extent(rate_dataset, yAccessor))
    .range([dimensions.boundedHeight, 0])
    .nice()

  const LT5Placement = yScale(5)
  const LT5 = bounds.append("rect")
        .attr("x", 0)
        .attr("width", dimensions.boundedWidth)
        .attr("y", LT5Placement)
        .attr("height", dimensions.boundedHeight
          - LT5Placement)
        .attr("fill", "#dcd7fa")

  bounds.append("text")
        .attr("class", "watermark")
        .attr("x", dimensions.width - 180)
        .attr("y", dimensions.height - 60)
        .attr("fill", "grey")
        .style("opacity", 0.5)
        .html("@Alephthoughts")
  
  bounds.append("defs").append("clipPath")
        .attr("id", "bounds-clip-path")
      .append("rect")
        .attr("width", dimensions.boundedWidth)
        .attr("height", dimensions.boundedHeight)
  
    const clip = bounds.append("g")
      .attr("clip-path", "url(#bounds-clip-path)")
  
  const xScale = d3.scaleTime()
    .domain(d3.extent(rate_dataset, xAccessor))
    .range([0, dimensions.boundedWidth])
    .nice()
    

  const lineGenerator = d3.line()
    .x(d => xScale(xAccessor(d)))
    .y(d => yScale(yAccessor(d)))
    .curve(d3.curveMonotoneX)

  const line = clip.append("path")
      .attr("class", "rate-line")
      .attr("d", lineGenerator(rate_dataset))


  const yAxisGenerator = d3.axisLeft()
      .scale(yScale)
      
  
  const yAxis = bounds.append("g")
      .attr("class", "y-axis")
      .call(yAxisGenerator)

  const yAxisLabel = yAxis.append("text")
      .attr("class", "y-axis-label")
      .attr("x", -dimensions.boundedHeight / 2)
      .attr("y", -dimensions.margin.left + 20)
      .html("Growth Rate in (%)")

      const xAxisGenerator = d3.axisBottom()
      .scale(xScale)
      .tickFormat(format)
      .ticks(d3.timeSunday)
      
    
  
  const xAxis = bounds.append("g")
      .attr("class", "x-axis")
      .style("transform", `translateY(${dimensions.boundedHeight}px)`)
      .call(xAxisGenerator)

  const listeningRect = bounds.append("rect")
      .attr("class", "listening-rect")
      .attr("width", dimensions.boundedWidth)
      .attr("height", dimensions.boundedHeight)
      .on("mousemove", onMouseMove)
      .on("mouseleave", onMouseLeave)

  const tooltip = d3.select("#tooltip")
  const tooltipCircle = bounds.append("circle")
          .attr("class", "tooltip-circle")
          .attr("r", 4)
          .attr("stroke", "#8B0000")
          .attr("fill", "white")
          .attr("stroke-width", 2)
          .style("opacity", 0)

  const lockdown = bounds.append("circle")
            .attr("cx", xScale(dateParser("2020-03-25")))
            .attr("cy", yScale(16.76300578034682))
            .attr("r", 4)
            .attr("fill", "#FF0D86")
            .attr("tabindex", "0")

  const lockdownLabel = bounds.append("text")
            .attr("class", "lockdown-annotation")
            .attr("x", xScale(dateParser("2020-03-25")))
            .attr("y", yScale(16.76300578034682) -10)
            .attr("fill", "#FF0D86")
            .attr("font-size", "0.7em")
            .attr("font-family", "Bradley Hand")
            .attr("text-anchor", "middle")
            .text("Lockdown")
            
      
  

  function onMouseMove() {
    const mousePosition = d3.mouse(this)
    const hoveredDate = xScale.invert(mousePosition[0])
        
    const getDistanceFromHoveredDate = d => Math.abs(xAccessor(d) - hoveredDate)
    const closestIndex = d3.scan(rate_dataset, (a, b) => (
    getDistanceFromHoveredDate(a) - getDistanceFromHoveredDate(b)
    ))
    const closestDataPoint = rate_dataset[closestIndex]
        
    const closestXValue = xAccessor(closestDataPoint)
    const closestYValue = yAccessor(closestDataPoint)
        
    const formatDate = d3.timeFormat("%A %B %-d, %Y")
    tooltip.select("#date")
    .text(formatDate(closestXValue))

    const formatRate = d => `${d3.format(".2f")(d)}%`
    tooltip.select("#rate")
        .html(formatRate(closestYValue))

    const x = xScale(closestXValue)
      + dimensions.margin.left
    const y = yScale(closestYValue)
      + dimensions.margin.top

    tooltip.style("transform", `translate(`
      + `calc(-50% + ${x}px),`
      + `calc(-120% + ${y}px)`
      + `)`)

    tooltip.style("opacity", 1)

    tooltipCircle
        .attr("cx", xScale(closestXValue))
        .attr("cy", yScale(closestYValue))
        .style("opacity", 1)
  }
  function onMouseLeave() {
    tooltip.style("opacity", 0)

    tooltipCircle.style("opacity", 0)
  } 
      
}
drawRateChart();
'''

In [25]:
#hide
html_rate_text = rate_html_temp.substitute({
    'css_text': css_text,
    'd3_rate_script' : d3_rate_script,
    'rate_dataset':rate_data_js
})

In [26]:
#hide_input
HTML(html_rate_text)

The above chart tracks % change in the number of cases on a given day.

The shaded region shows the number of days when the confirmed cases grew by less than 5%.

The pink datapoint marks the beginning of the government initiated lockdown of 1.3 billion Indian population.

## State-wise Tally

In [27]:
#hide
table_data = data[-2:]

In [28]:
#hide
state = []
confirmed = []
domestic = []
foreign = []
recovered = []
deceased = []
change = []
for item in table_data[1]['regional']:
    if item['totalConfirmed'] > 0:
        for item2 in table_data[0]['regional']:
            if item['loc'] == item2['loc']:
                state.append(item['loc'])
                confirmed.append(item['totalConfirmed'])
                domestic.append(item['confirmedCasesIndian'])
                foreign.append(item['confirmedCasesForeign'])
                recovered.append(item['discharged'])
                deceased.append(item['deaths'])
                change.append(item['totalConfirmed'] - item2['totalConfirmed'])

state_data = json.dumps([{"state": state, "confirmed": confirmed, "domestic": domestic, "foreign": foreign
            , "recovered": recovered, "deceased": deceased, "change": change} for state, confirmed
            , domestic, foreign, recovered, deceased, change in zip(state, confirmed, domestic, foreign, recovered, deceased, change)])

In [29]:
#hide
table_data_js = f"let table_dataset = {state_data} "

>Note: All percentages are calculated based on the number of confirmed cases.

In [30]:
#hide
table_html_temp = Template('''
    <script src = "https://d3js.org/d3.v5.min.js"></script> 
    <style scoped>
        $table_css_text
    </style>
    <table id="chart-table">
    </table>
    <script>
    $table_dataset
    $d3_table_script
    </script>
''')
            

In [31]:
#hide
table_css_text = '''
@font-face {
    font-family: 'Inter var';
    src: url('https://rsms.me/inter/inter.css');
}

chart-body {
    display: flex;
    flex-direction: column;
    align-items: center;
    padding: 5em 2em;
    letter-spacing: -0.011em;
    font-family: 'Inter var', sans-serif;
    font-size: 16px;
    color: #34495e;
    background: #f8f9fa;
}

chart-table {
    border-collapse: collapse;
}

thead {
    font-size: 0.9em;
    letter-spacing: 0;
    font-weight: 900;
    white-space: nowrap;
    text-align: left;
}

thead th {
    position: sticky;
    top: 0;
    color: #f8f9fa;
    background: #34495e;
    padding: 1em 0.8rem;
}

tbody tr:nth-child(odd) {
    background: #f0f0f3;
}

tbody tr:hover {
    background: #d7ecec;
}

td {
    padding: 0.8em 0.8rem;
}

.number {
    text-align: right;
    font-feature-settings: 'tnum' 1;
}

.centered {
    text-align: center;
}

tbody .text {
    font-size: 0.9em;
    white-space: nowrap;
    text-overflow: ellipsis;
    overflow: hidden;
}
'''

In [32]:
#hide
d3_table_script = """
function drawTable(){

    function zerofy(d){
        if (isNaN(d)){
            return d3.format(".2f")(0)
        }
        return d
    }
    const numberOfRows = table_dataset.length
    const table = d3.select("#chart-table")
    const colorScale = d3.interpolateHcl("#cde8a4", "#efa8a1")
    const rcolorScale = d3.interpolateHcl("#efa8a1","#cde8a4")
    const gcolorScale = d3.interpolateHcl("#fcfcfc","#212020")
    const confirmedScale = d3.scaleLinear()
    .domain(d3.extent(table_dataset.slice(0, numberOfRows), d => d.confirmed))
    .range([0, 1])
    const changeScale = d3.scaleLinear()
    .domain(d3.extent(table_dataset.slice(0, numberOfRows), d => d.change))
    .range([0, 1])
    const recoveredScale = d3.scaleLinear()
    .domain(d3.extent(table_dataset.slice(0, numberOfRows), d => (d.recovered/d.confirmed)*100))
    .range([0, 1])
    const mortalityScale = d3.scaleLinear()
    .domain(d3.extent(table_dataset.slice(0, numberOfRows), d => (d.deceased/d.confirmed)*100))
    .range([0, 1])

    

    const columns = [
        {"label" : "State", "type" : "text", "format" : d => d.state}
       ,{"label" : "Confirmed Cases", "type" : "number", "format" : d => d.confirmed, background: d => colorScale(confirmedScale(d.confirmed))}
       ,{"label" : "New Cases", "type" : "number", "format" : d => d.change, background: d => colorScale(changeScale(d.change))} 
       ,{"label" : "Recovered", "type" : "number", "format" : d => d.recovered} 
       ,{"label" : "Recovery %", "type" : "number", "format" : d => `${zerofy((d3.format(".2f") ((d.recovered/d.confirmed)*100)))}%`, background: d => rcolorScale(recoveredScale((d.recovered/d.confirmed)*100))} 
       ,{"label" : "Deceased", "type" : "number", "format" : d => d.deceased} 
       ,{"label" : "Mortality %", "type" : "number", "format" : d => `${zerofy((d3.format(".2f") ((d.deceased/d.confirmed)*100)))}%`, background: d => gcolorScale(recoveredScale(zerofy((d.deceased/d.confirmed)*100)))} 
    ]

    table.append("thead").append("tr")
    .selectAll("thead")
    .data(columns)
    .enter().append("th")
      .text(d => d.label)
      .attr("class", d => d.type)

    const chart_body = table.append("tbody")

    table_dataset.slice(0, numberOfRows).forEach(d => {
        chart_body.append("tr")
          .selectAll("td")
          .data(columns)
          .enter().append("td")
            .text(column => column.format(d))
            .attr("class", column => column.type)
            .style("background", column => column.background && column.background(d))
            .style("transform", column => column.transform && column.transform(d))
      })
}
drawTable();
"""

In [33]:
#hide
html_table_text = table_html_temp.substitute({
    'table_css_text': table_css_text,
    'd3_table_script' : d3_table_script,
    'table_dataset':table_data_js
})

In [34]:
#hide_input
HTML(html_table_text)

Covid-19 free states: Sikkim{% fn 2 %}, Nagaland{% fn 2 %}, Arunachal Pradesh, Goa, Manipur and Tripura.

## State-wise Growth Trajectories

In [35]:
#hide
stateT_raw = requests.get("https://api.rootnet.in/covid19-in/stats/history").text
stateT = json.loads(stateT_raw)['data']

In [36]:
#hide
list_of_states = ["Andaman and Nicobar Islands", "Andhra Pradesh", "Arunachal Pradesh", "Assam", "Bihar", "Chandigarh", "Chhattisgarh",
                  "Delhi", "Goa", "Gujarat", "Haryana", "Himachal Pradesh", "Jammu and Kashmir", "Jharkhand", "Karnataka", "Kerala",
                  "Ladakh", "Madhya Pradesh", "Maharashtra", "Manipur", "Meghalaya", "Mizoram", "Nagaland#", "Odisha", "Puducherry", 
                  "Punjab", "Rajasthan", "Tamil Nadu", "Telengana", "Tripura", "Uttarakhand", "Uttar Pradesh", "West Bengal"]

In [37]:
#hide
AndamanandNicobarIslands = [{"Day": 0, "Confirmed":0}]
AndhraPradesh = [{"Day": 0, "Confirmed":0}]
ArunachalPradesh = [{"Day": 0, "Confirmed":0}]
Assam = [{"Day": 0, "Confirmed":0}]
Bihar = [{"Day": 0, "Confirmed":0}]
Chandigarh = [{"Day": 0, "Confirmed":0}]
Chhattisgarh = [{"Day": 0, "Confirmed":0}]
Delhi = [{"Day": 0, "Confirmed":0}]
Goa = [{"Day": 0, "Confirmed":0}]
Gujarat = [{"Day": 0, "Confirmed":0}]
Haryana = [{"Day": 0, "Confirmed":0}]
HimachalPradesh = [{"Day": 0, "Confirmed":0}]
JammuandKashmir = [{"Day": 0, "Confirmed":0}]
Jharkhand = [{"Day": 0, "Confirmed":0}]
Karnataka = [{"Day": 0, "Confirmed":0}]
Kerala = [{"Day": 0, "Confirmed":0}]
Ladakh = [{"Day": 0, "Confirmed":0}]
MadhyaPradesh = [{"Day": 0, "Confirmed":0}]
Maharashtra = [{"Day": 0, "Confirmed":0}]
Manipur = [{"Day": 0, "Confirmed":0}]
Meghalaya = [{"Day": 0, "Confirmed":0}]
Mizoram = [{"Day": 0, "Confirmed":0}]
Nagaland = [{"Day": 0, "Confirmed":0}]
Odisha = [{"Day": 0, "Confirmed":0}]
Puducherry = [{"Day": 0, "Confirmed":0}]
Punjab = [{"Day": 0, "Confirmed":0}]
Rajasthan = [{"Day": 0, "Confirmed":0}]
TamilNadu = [{"Day": 0, "Confirmed":0}]
Telengana = [{"Day": 0, "Confirmed":0}]
Tripura = [{"Day": 0, "Confirmed":0}]
Uttarakhand = [{"Day": 0, "Confirmed":0}]
UttarPradesh = [{"Day": 0, "Confirmed":0}]
WestBengal = [{"Day": 0, "Confirmed":0}]

state_list_name = [AndamanandNicobarIslands, AndhraPradesh, ArunachalPradesh, Assam, Bihar, Chandigarh, Chhattisgarh,
                  Delhi, Goa, Gujarat, Haryana, HimachalPradesh, JammuandKashmir, Jharkhand, Karnataka, Kerala,
                  Ladakh, MadhyaPradesh, Maharashtra, Manipur, Meghalaya, Mizoram, Nagaland, Odisha, Puducherry, 
                  Punjab, Rajasthan, TamilNadu, Telengana, Tripura, Uttarakhand, UttarPradesh, WestBengal]

def add_data(state_list_name, state_name):
    d = 1
    for record in stateT:
        for item in record["regional"]:
                if (item["loc"] == state_name) and (item["totalConfirmed"]>500):
                    state_list_name.append({"Day": d, "Confirmed":item["totalConfirmed"]})
                    d += 1

In [38]:
#hide
for state_list, state in zip(state_list_name, list_of_states):
    add_data(state_list, state)


In [39]:
#hide
comparable_data = []
for d in range(Maharashtra[-1]['Day'] + 7):
    comparable_data.append({"Day": d, "Confirmed": d*500})

In [40]:
#hide
js_mh_dataset = f"let mh_dataset = {json.dumps(Maharashtra)};"
js_ap_dataset = f"let ap_dataset = {json.dumps(AndhraPradesh)};"
js_bi_dataset = f"let bi_dataset = {json.dumps(Bihar)};"
js_de_dataset = f"let de_dataset = {json.dumps(Delhi)};"
js_gj_dataset = f"let gj_dataset = {json.dumps(Gujarat)};"
js_hy_dataset = f"let hy_dataset = {json.dumps(Haryana)};"
js_jk_dataset = f"let jk_dataset = {json.dumps(JammuandKashmir)};"
js_ka_dataset = f"let ka_dataset = {json.dumps(Karnataka)};"
js_kl_dataset = f"let kl_dataset = {json.dumps(Kerala)};"
js_mp_dataset = f"let mp_dataset = {json.dumps(MadhyaPradesh)};"
js_pj_dataset = f"let pj_dataset = {json.dumps(Punjab)};"
js_rj_dataset = f"let rj_dataset = {json.dumps(Rajasthan)};"
js_tn_dataset = f"let tn_dataset = {json.dumps(TamilNadu)};"
js_tg_dataset = f"let tg_dataset = {json.dumps(Telengana)};"
js_up_dataset = f"let up_dataset = {json.dumps(UttarPradesh)};"
js_wb_dataset = f"let wb_dataset = {json.dumps(WestBengal)};"
js_comp_dataset = f"let comp_dataset = {json.dumps(comparable_data)};"

In [41]:
#hide
st_html_temp = Template('''
    <script src = "https://d3js.org/d3.v5.min.js"></script> 
    <style scoped>
        $st_css_text
    </style>
    <div id="st-wrapper" class="st-wrapper">
    </div>
    <script>
    $mh
    $ap
    $bi
    $de
    $gj
    $hy
    $jk
    $ka
    $kl
    $mp
    $pj
    $rj
    $tn
    $tg
    $up
    $wb
    $comp
    $d3_st_script
    </script>
''')

In [42]:
#hide
st_css_text = '''
.st-wrapper {
    position: relative;
}

.y-axis-label {
    fill: gray;
    font-size: 1.4em;
    text-anchor: middle;
    transform: rotate(-90deg);
}

.x-axis-label {
    fill: gray;
    font-size: 1.4em;
    text-anchor: middle;
}

.x-axis {
    color: gray;
    font: 10px Georgia;
    font-weight: bold;
    
  }
  
.y-axis path,
.y-axis line {
    fill: none;
    stroke: gray;
    shape-rendering: crispEdges;
    stroke-width: 2px;
  }

.y-axis {
    color: gray;
    font: 10px sans-serif;
    font-weight: bold;
    
}
  
  .x-axis path,
  .x-axis line {
    fill: none;
    stroke: gray;
    shape-rendering: crispEdges;
    stroke-width: 2px;
  }

.y-axisR path,
.y-axisR line {
      fill: none;
      stroke: gray;
      shape-rendering: crispEdges;
      stroke-width: 2px;
    }
  
.y-axisR {
      color: gray;
      font: 10px sans-serif;
      font-weight: bold;
      
}
.comp-line {
    fill: none;
    stroke:gray;
    stroke-width: 2;
    stroke-dasharray: 3, 3;
}

.comp-text {
    fill:	gray;
    font-family: Georgia;
    font-weight: bold;
    text-anchor: middle;
    transform: rotate(-24deg)
}

.ap-line {
    fill: none;
    stroke:#CCCC00;
    stroke-width: 2;
}

.ap-circle {
    fill:#CCCC00;
    stroke: #000000;
}

.ap-text {
    fill:	#CCCC00;
    font-family: Georgia;
    font-weight: bold;
}

.mh-line {
    fill: none;
    stroke: #FF9933;
    stroke-width: 2;
}

.mh-circle {
    fill: #FF9933;
    stroke: #000000;
}

.mh-text {
    fill: #FF9933;
    font-family: Georgia;
    font-weight: bold;
}

.bi-line {
    fill: none;
    stroke: #064771;
    stroke-width: 2;
}

.bi-circle {
    fill: #064771;
    stroke: #000000;
}

.bi-text {
    fill: #064771;
    font-family: Georgia;
    font-weight: bold;
}

.de-line {
    fill: none;
    stroke: #006400;
    stroke-width: 2;
}

.de-circle {
    fill: #006400;
    stroke: #000000;
}

.de-text {
    fill: #006400;
    font-family: Georgia;
    font-weight: bold;
}

.gj-line {
    fill: none;
    stroke: #DC143C;
    stroke-width: 2;
}

.gj-circle {
    fill: #DC143C;
    stroke: #000000;
}

.gj-text {
    fill: #DC143C;
    font-family: Georgia;
    font-weight: bold;
}

.hy-line {
    fill: none;
    stroke: #6AAB8E;
    stroke-width: 2;
}

.hy-circle {
    fill: #6AAB8E;
    stroke: #000000;
}

.jk-text {
    fill: #FF0000;
    font-family: Georgia;
    font-weight: bold;
}

.jk-line {
    fill: none;
    stroke: #FF0000;
    stroke-width: 2;
}

.jk-circle {
    fill: #FF0000;
    stroke: #000000;
}

.hy-text {
    fill: #6AAB8E;
    font-family: Georgia;
    font-weight: bold;
}

.ka-line {
    fill: none;
    stroke: #FFACEC;
    stroke-width: 2;
}

.ka-circle {
    fill: #FFACEC;
    stroke: #000000;
}

.ka-text {
        fill: #FFACEC;
        font-family: Georgia;
        font-weight: bold;
    }

.kl-line {
        fill: none;
        stroke: #6495ed;
        stroke-width: 2;
    }
    
.kl-circle {
        fill: #6495ed;
        stroke: #000000;
    }
    
.kl-text {
            fill: #6495ed;
            font-family: Georgia;
            font-weight: bold;
        }
.mp-line {
            fill: none;
            stroke: #db9900;
            stroke-width: 2;
        }
        
.mp-circle {
            fill: #db9900;
            stroke: #000000;
        }
        
.mp-text {
                fill: #db9900;
                font-family: Georgia;
                font-weight: bold;
            }

.pj-line {
                fill: none;
                stroke: #800000;
                stroke-width: 2;
            }
            
.pj-circle {
                fill: #800000;
                stroke: #000000;
            }
            
.pj-text {
                    fill: #800000;
                    font-family: Georgia;
                    font-weight: bold;
                }
.rj-line {
                    fill: none;
                    stroke: #59DF00;
                    stroke-width: 2;
                }
                
.rj-circle {
                    fill: #59DF00;
                    stroke: #000000;
                }
.rj-text {
                        fill: #59DF00;
                        font-family: Georgia;
                        font-weight: bold;
                    }

.tn-line {
                        fill: none;
                        stroke: #5757FF;
                        stroke-width: 2;
                    }
                    
.tn-circle {
                        fill: #5757FF;
                        stroke: #000000;
                    }
.tn-text {
                            fill: #5757FF;
                            font-family: Georgia;
                            font-weight: bold;
                        }

.tg-line {
                            fill: none;
                            stroke: #23819C;
                            stroke-width: 2;
                        }
                        
.tg-circle {
                            fill: #23819C;
                            stroke: #000000;
                        }
.tg-text {
                                fill: #23819C;
                                font-family: Georgia;
                                font-weight: bold;
                            }

.up-line {
                                fill: none;
                                stroke: #FF4848;
                                stroke-width: 2;
                            }
                            
.up-circle {
                                fill: #FF4848;
                                stroke: #000000;
                            }
.up-text {
                                    fill: #FF4848;
                                    font-family: Georgia;
                                    font-weight: bold;
                                }

.wb-line {
                                    fill: none;
                                    stroke: #404040;
                                    stroke-width: 2;
                                }
                                
.wb-circle {
                                    fill: #404040;
                                    stroke: #000000;
                                }
.wb-text {
                                        fill: #404040;
                                        font-family: Georgia;
                                        font-weight: bold;
                                    }

.notes{
    fill: gray;
    font-family: Georgia;
    font-size: 0.8em;
}



.stchartcon {
    display: flex;
    justify-content: center;
    padding: 5em 1em;
    font-family: sans-serif;
}
'''

In [43]:
#hide
d3_st_script = '''
function drawStateTrajectory() {
  const yAccessor = d => d.Confirmed
  const xAccessor = d => d.Day

  let st_dimensions = {
    width: 750,
    height: 580,
    margin: {
      top: 20,
      right: 70,
      bottom: 70,
      left: 70,
    },
  }
  st_dimensions.boundedWidth = st_dimensions.width - st_dimensions.margin.left - st_dimensions.margin.right
  st_dimensions.boundedHeight = st_dimensions.height - st_dimensions.margin.top - st_dimensions.margin.bottom

  // 3. Draw canvas

  const st_wrapper = d3.select("#st-wrapper")
    .append("svg")
    .attr("preserveAspectRatio", "xMinYMin meet")
    .attr("viewBox", "0 0 750 580")
    //.attr("width", st_dimensions.width)
    //.attr("height", st_dimensions.height)

  st_wrapper.append("rect")
    .attr("class", "background")
    .attr("width", "100%")
    .attr("height", "100%")
    .attr("fill", "#faebd7")

  const st_bounds = st_wrapper.append("g")
      .attr("transform", `translate(${
        st_dimensions.margin.left
      }, ${
        st_dimensions.margin.top
      })`)

  st_bounds.append("text")
      .attr("class", "watermark")
      .attr("x", st_dimensions.width - 250)
      .attr("y", st_dimensions.height - 100)
      .attr("fill", "grey")
      .style("opacity", 0.5)
      .html("@Alepthoughts")

  st_bounds.append("defs").append("clipPath")
      .attr("id", "bounds-clip-path")
    .append("rect")
      .attr("width", st_dimensions.boundedWidth)
      .attr("height", st_dimensions.boundedHeight)

  const clip = st_bounds.append("g")
    .attr("clip-path", "url(#bounds-clip-path)")

  // 4. Create scales

  const yScale = d3.scaleLinear()
    .domain(d3.extent(mh_dataset, yAccessor))
    .range([st_dimensions.boundedHeight, 0])
    .nice()


  
  const xScale = d3.scaleLinear()
    .domain([0, d3.extent(mh_dataset, xAccessor)[1]+ 7])
    .range([0, st_dimensions.boundedWidth])
    .nice()
      
  const yAxisGenerator = d3.axisLeft()
      .scale(yScale)
      
      

  const yAxis = st_bounds.append("g")
      .attr("class", "y-axis")
      .call(yAxisGenerator)

  const yAxisGeneratorR = d3.axisRight()
      .scale(yScale)

  const yAxisR = st_bounds.append("g")
      .attr("class", "y-axisR")
      .style("transform", `translateX(${st_dimensions.boundedWidth}px)`)
      .call(yAxisGeneratorR)

  const xAxisGenerator = d3.axisBottom()
      .scale(xScale)
  
  const xAxis = st_bounds.append("g")
      .attr("class", "x-axis")
      .style("transform", `translateY(${st_dimensions.boundedHeight}px)`)
      .call(xAxisGenerator)
      


  const yAxisLabel = yAxis.append("text")
      .attr("class", "y-axis-label")
      .attr("x", -st_dimensions.boundedHeight / 2)
      .attr("y", -st_dimensions.margin.left + 20)
      .html("Confirmed Cases")

  const xAxisLabel = xAxis.append("text")
      .attr("class", "x-axis-label")
      .attr("x", st_dimensions.boundedWidth/ 2)
      .attr("y", st_dimensions.boundedHeight - 460)
      .html("Days since confirmed cases passed 500.")

const lineGenerator = d3.line()
      .x(d => xScale(xAccessor(d)))
      .y(d => yScale(yAccessor(d)))

// COMP

const line_COMP = clip.append("path")
        .attr("class", "comp-line")
        .attr("d", lineGenerator(comp_dataset))
  
/*const anno_COMP = st_bounds.append("text")
        .attr("class", "comp-text")
        .attr("x", (st_dimensions.boundedWidth * 0.75) - 20)
        .attr("y", (st_dimensions.boundedHeight * 0.75) + 200)*/
        .html("500 Cases Per Day")
     
//Maharashtra
const line_MH = clip.append("path")
        .attr("class", "mh-line")
        .attr("d", lineGenerator(mh_dataset))
  
const circle_MH = st_bounds.append("circle")
        .attr("class", "mh-circle")
        .attr("cx", xScale(xAccessor(mh_dataset[mh_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(mh_dataset[mh_dataset.length - 1])))
        .attr("r", 3)
  
const anno_MH = st_bounds.append("text")
        .attr("class", "mh-text")
        .attr("x", xScale(xAccessor(mh_dataset[mh_dataset.length - 1])) + 5)
        .attr("y", yScale(yAccessor(mh_dataset[mh_dataset.length - 1])))
        .html("Maharashtra")

// Kerala

const line_KL = clip.append("path")
        .attr("class", "kl-line")
        .attr("d", lineGenerator(kl_dataset))
  
const circle_KL = st_bounds.append("circle")
        .attr("class", "kl-circle")
        .attr("cx", xScale(xAccessor(kl_dataset[kl_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(kl_dataset[kl_dataset.length - 1])))
        .attr("r", 3)
  
const anno_KL = st_bounds.append("text")
        .attr("class", "kl-text")
        .attr("x", xScale(xAccessor(kl_dataset[kl_dataset.length - 1])) + 5)
        .attr("y", yScale(yAccessor(kl_dataset[kl_dataset.length - 1])))
        .html("Kerala")

// Karnataka

const line_KA = clip.append("path")
        .attr("class", "ka-line")
        .attr("d", lineGenerator(ka_dataset))
  
const circle_KA = st_bounds.append("circle")
        .attr("class", "ka-circle")
        .attr("cx", xScale(xAccessor(ka_dataset[ka_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(ka_dataset[ka_dataset.length - 1])))
        .attr("r", 3)
  
const anno_KA = st_bounds.append("text")
        .attr("class", "ka-text")
        .attr("x", xScale(xAccessor(ka_dataset[ka_dataset.length - 1])) + 5)
        .attr("y", yScale(yAccessor(ka_dataset[ka_dataset.length - 1])))
        .html("Karnataka")

// Tamil Nadu

const line_TN = clip.append("path")
        .attr("class", "tn-line")
        .attr("d", lineGenerator(tn_dataset))
  
const circle_TN = st_bounds.append("circle")
        .attr("class", "tn-circle")
        .attr("cx", xScale(xAccessor(tn_dataset[tn_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(tn_dataset[tn_dataset.length - 1])))
        .attr("r", 3)
  
const anno_TN = st_bounds.append("text")
        .attr("class", "tn-text")
        .attr("x", xScale(xAccessor(tn_dataset[tn_dataset.length - 1])))
        .attr("y", yScale(yAccessor(tn_dataset[tn_dataset.length - 1])))
        .html("Tamil Nadu")

// Delhi

const line_DE = clip.append("path")
        .attr("class", "de-line")
        .attr("d", lineGenerator(de_dataset))
  
const circle_DE = st_bounds.append("circle")
        .attr("class", "de-circle")
        .attr("cx", xScale(xAccessor(de_dataset[de_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(de_dataset[de_dataset.length - 1])))
        .attr("r", 3)
  
const anno_DE = st_bounds.append("text")
        .attr("class", "de-text")
        .attr("x", xScale(xAccessor(de_dataset[de_dataset.length - 1])) + 5)
        .attr("y", yScale(yAccessor(de_dataset[de_dataset.length - 1])))
        .html("Delhi")

// Rajasthan

const line_RJ = clip.append("path")
        .attr("class", "rj-line")
        .attr("d", lineGenerator(rj_dataset))
  
const circle_RJ = st_bounds.append("circle")
        .attr("class", "rj-circle")
        .attr("cx", xScale(xAccessor(rj_dataset[rj_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(rj_dataset[rj_dataset.length - 1])))
        .attr("r", 3)
  
const anno_RJ = st_bounds.append("text")
        .attr("class", "rj-text")
        .attr("x", xScale(xAccessor(rj_dataset[rj_dataset.length - 1])))
        .attr("y", yScale(yAccessor(rj_dataset[rj_dataset.length - 1]))- 10)
        .html("Rajasthan")

// Telangana

const line_TG = clip.append("path")
        .attr("class", "tg-line")
        .attr("d", lineGenerator(tg_dataset))
  
const circle_Tg = st_bounds.append("circle")
        .attr("class", "tg-circle")
        .attr("cx", xScale(xAccessor(tg_dataset[tg_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(tg_dataset[tg_dataset.length - 1])))
        .attr("r", 3)
  
const anno_TG = st_bounds.append("text")
        .attr("class", "tg-text")
        .attr("x", xScale(xAccessor(tg_dataset[tg_dataset.length - 1])) + 5)
        .attr("y", yScale(yAccessor(tg_dataset[tg_dataset.length - 1])))
        .html("Telangana")

// Andhra Pradesh

const line_AP = clip.append("path")
        .attr("class", "ap-line")
        .attr("d", lineGenerator(ap_dataset))
  
const circle_AP = st_bounds.append("circle")
        .attr("class", "ap-circle")
        .attr("cx", xScale(xAccessor(ap_dataset[ap_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(ap_dataset[ap_dataset.length - 1])))
        .attr("r", 3)
  
const anno_AP = st_bounds.append("text")
        .attr("class", "ap-text")
        .attr("x", xScale(xAccessor(ap_dataset[ap_dataset.length - 1])) + 5)
        .attr("y", yScale(yAccessor(ap_dataset[ap_dataset.length - 1])))
        .html("Andhra Pradesh")

// Madhya Pradesh

const line_MP = clip.append("path")
        .attr("class", "mp-line")
        .attr("d", lineGenerator(mp_dataset))
  
const circle_MP = st_bounds.append("circle")
        .attr("class", "mp-circle")
        .attr("cx", xScale(xAccessor(mp_dataset[mp_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(mp_dataset[mp_dataset.length - 1])))
        .attr("r", 3)
  
const anno_MP = st_bounds.append("text")
        .attr("class", "mp-text")
        .attr("x", xScale(xAccessor(mp_dataset[mp_dataset.length - 1])) + 5)
        .attr("y", yScale(yAccessor(mp_dataset[mp_dataset.length - 1])))
        .html("MP")

// Gujarat

const line_GJ = clip.append("path")
        .attr("class", "gj-line")
        .attr("d", lineGenerator(gj_dataset))
  
const circle_GJ = st_bounds.append("circle")
        .attr("class", "gj-circle")
        .attr("cx", xScale(xAccessor(gj_dataset[gj_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(gj_dataset[gj_dataset.length - 1])))
        .attr("r", 3)
  
const anno_GJ = st_bounds.append("text")
        .attr("class", "gj-text")
        .attr("x", xScale(xAccessor(gj_dataset[gj_dataset.length - 1])) + 5)
        .attr("y", yScale(yAccessor(gj_dataset[gj_dataset.length - 1])))
        .html("Gujarat")

// Jammu and Kashmir

const line_JK = clip.append("path")
        .attr("class", "jk-line")
        .attr("d", lineGenerator(jk_dataset))
  
const circle_JK = st_bounds.append("circle")
        .attr("class", "jk-circle")
        .attr("cx", xScale(xAccessor(jk_dataset[jk_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(jk_dataset[jk_dataset.length - 1])))
        .attr("r", 3)
  
const anno_JK = st_bounds.append("text")
        .attr("class", "jk-text")
        .attr("x", xScale(xAccessor(jk_dataset[jk_dataset.length - 1])) + 5)
        .attr("y", yScale(yAccessor(jk_dataset[jk_dataset.length - 1])))
        .html("J&K")

// Haryana

const line_HY = clip.append("path")
        .attr("class", "hy-line")
        .attr("d", lineGenerator(hy_dataset))
  
const circle_HY = st_bounds.append("circle")
        .attr("class", "hy-circle")
        .attr("cx", xScale(xAccessor(hy_dataset[hy_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(hy_dataset[hy_dataset.length - 1])))
        .attr("r", 3)
  
const anno_HY = st_bounds.append("text")
        .attr("class", "hy-text")
        .attr("x", xScale(xAccessor(hy_dataset[hy_dataset.length - 1])) + 5)
        .attr("y", yScale(yAccessor(hy_dataset[hy_dataset.length - 1]))+ 10)
        .html("Haryana")

// Punjab

const line_PJ = clip.append("path")
        .attr("class", "pj-line")
        .attr("d", lineGenerator(pj_dataset))
  
const circle_PJ = st_bounds.append("circle")
        .attr("class", "pj-circle")
        .attr("cx", xScale(xAccessor(pj_dataset[pj_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(pj_dataset[pj_dataset.length - 1])))
        .attr("r", 3)
  
const anno_PJ = st_bounds.append("text")
        .attr("class", "pj-text")
        .attr("x", xScale(xAccessor(pj_dataset[pj_dataset.length - 1]))- 5)
        .attr("y", yScale(yAccessor(pj_dataset[pj_dataset.length - 1]))+ 20)
        .html("Punjab")

// Bihar

const line_BI = clip.append("path")
        .attr("class", "bi-line")
        .attr("d", lineGenerator(bi_dataset))
  
const circle_BI = st_bounds.append("circle")
        .attr("class", "bi-circle")
        .attr("cx", xScale(xAccessor(bi_dataset[bi_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(bi_dataset[bi_dataset.length - 1])))
        .attr("r", 3)
  
const anno_BI = st_bounds.append("text")
        .attr("class", "bi-text")
        .attr("x", xScale(xAccessor(bi_dataset[bi_dataset.length - 1])) + 5)
        .attr("y", yScale(yAccessor(bi_dataset[bi_dataset.length - 1]))-10)
        .html("Bihar")

// UP

const line_UP = clip.append("path")
        .attr("class", "up-line")
        .attr("d", lineGenerator(up_dataset))
  
const circle_UP = st_bounds.append("circle")
        .attr("class", "up-circle")
        .attr("cx", xScale(xAccessor(up_dataset[up_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(up_dataset[up_dataset.length - 1])))
        .attr("r", 3)
  
const anno_UP = st_bounds.append("text")
        .attr("class", "up-text")
        .attr("x", xScale(xAccessor(up_dataset[up_dataset.length - 1])) + 5)
        .attr("y", yScale(yAccessor(up_dataset[up_dataset.length - 1])))
        .html("Uttar Pradesh")

// WB

const line_WB = clip.append("path")
        .attr("class", "wb-line")
        .attr("d", lineGenerator(wb_dataset))
  
const circle_WB = st_bounds.append("circle")
        .attr("class", "wb-circle")
        .attr("cx", xScale(xAccessor(wb_dataset[wb_dataset.length - 1])))
        .attr("cy", yScale(yAccessor(wb_dataset[wb_dataset.length - 1])))
        .attr("r", 3)
  
const anno_WB = st_bounds.append("text")
        .attr("class", "wb-text")
        .attr("x", xScale(xAccessor(wb_dataset[wb_dataset.length - 1])) + 5)
        .attr("y", yScale(yAccessor(wb_dataset[wb_dataset.length - 1])))
        .html("WB")

st_bounds.selectAll(".tick")
        .filter(function (d) { return d === 0;  })
        .remove();

format = d3.timeFormat("%A,%B %d, %I:%M %p")
st_wrapper.append("text")
      .attr("class", "notes")
      .attr("x", 0)
      .attr("y", st_dimensions.height - 20)
      .html("alephthoughts.com :- Abhishek Sharma / @alephthoughts")
st_wrapper.append("text")
      .attr("class", "notes")
      .attr("x", 0)
      .attr("y", st_dimensions.height - 5)
      .html("source: https://api.rootnet.in/. Chart Updated June 8, 13:25 IST.")


}
drawStateTrajectory();
'''

In [44]:
#hide
html_st_text = st_html_temp.substitute({
    'st_css_text': st_css_text,
    'd3_table_script' : d3_st_script,
    'mh': js_mh_dataset,
    'ap': js_ap_dataset,
    'bi':js_bi_dataset,
    'de': js_de_dataset,
    'gj': js_gj_dataset,
    'hy': js_hy_dataset,
    'jk': js_jk_dataset,
    'ka': js_ka_dataset,
    'kl': js_kl_dataset,
    'mp': js_mp_dataset,
    'pj': js_pj_dataset,
    'rj': js_rj_dataset,
    'tn': js_tn_dataset,
    'tg': js_tg_dataset,
    'up': js_up_dataset,
    'wb': js_wb_dataset,
    'comp': js_comp_dataset,
    'd3_st_script': d3_st_script
})

In [45]:
#hide_input
HTML(html_st_text)

* The rapid spread continues in Maharashtra. Delhi spread worsens.

## Testing Status

In [46]:
#hide
testing = requests.get("https://api.rootnet.in/covid19-in/stats/testing/latest").text
testing_day = datetime.strptime(json.loads(testing)['data']['day'], "%Y-%m-%d")

In [47]:
#hide_input
commentary.printbeautiful(f"""Total samples tested as of {testing_day.strftime("%A, %B, %d %Y")} : {json.loads(testing)['data']['totalSamplesTested']}.""")
if json.loads(testing)['data']['totalIndividualsTested']:
    commentary.printbeautiful(f"""Total individuals tested : {json.loads(testing)['data']['totalIndividualsTested']}.""")
if json.loads(testing)['data']['totalPositiveCases']:
    commentary.printbeautiful(f"""Total tests that resulted in positive : {json.loads(testing)['data']['totalPositiveCases']}.""")

<span style='color: #515151;
                        font-family: Helvetica Neue;
                        font-weight: 400;
                        font-size: 20px;
                        font-style: normal;
                        text-decoration: none;
                        letter-spacing: 1px;
                        line-height: 30px;
                        word-spacing: 1px;
                        text-shadow: none;
                        background-color: none;'>
                        Total samples tested as of Monday, June, 08 2020 : 4774434.
                        </span>
                        

In [48]:
#hide
lastRefreshedTimestamp = datetime.strptime(json.loads(data_raw)["lastRefreshed"][:-1],'%Y-%m-%dT%H:%M:%S.%f')
lastRefreshedTimestamp = lastRefreshedTimestamp.strftime("%A, %B %d, %I:%M %p")

In [49]:
#hide_input
end = BeautifulText(font_family="Helvetica Neue", color="#515151", font_size="12", font_weight=400)
end.printbeautiful(f"The data was last refreshed at {lastRefreshedTimestamp}.")

<span style='color: #515151;
                        font-family: Helvetica Neue;
                        font-weight: 400;
                        font-size: 12px;
                        font-style: normal;
                        text-decoration: none;
                        letter-spacing: 1px;
                        line-height: 0.8px;
                        word-spacing: 1px;
                        text-shadow: none;
                        background-color: none;'>
                        The data was last refreshed at Monday, June 08, 08:18 AM.
                        </span>
                        

{{ 'The official Ministry of Health and Familiy Welfare API can be found [here](https://api.rootnet.in/)' | fndetail : 1}}

{{ 'No Covid-19 cases registered in this state.'| fndetail : 2}}